<a href="https://colab.research.google.com/github/disha-cpu/Resources-For-Stat-Data/blob/master/4D_DBSCAN_seasonal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from plotnine import *
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import plotly.express as px
import plotly.graph_objects as go
import random
import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

In [112]:
# reading data
data = pd.read_csv('/content/weatherGOA.csv')
# formatting time
data = data.replace(to_replace ="24:00:00", value ="00:00")
data['time'] = data['Date'] +" "+ data['UT time'] 
data['time'] = pd.to_datetime(data['time'])
# converting temprature from K to C
data = data[data['time'].dt.minute == 0]
data['Temperature (C)'] = data['Temperature (K)'] - 273.15
data['Hour'] = data['time'].dt.hour.astype(float)
# 4D dataset
# dataset = data[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]



In [113]:
winter = data[data.time.dt.month == 1]
winter2 = data[data.time.dt.month == 2]
winter11 = data[data.time.dt.month == 11]
winter12 = data[data.time.dt.month == 12]
winter = winter.append(winter2, ignore_index = True)
winter = winter.append(winter12, ignore_index = True)
winter = winter.append(winter11, ignore_index = True)
winter_dataset = winter[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]
winter_copy = winter[['time',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]]
winter_copy = winter_copy.sort_values(by=['time'])

In [ ]:
winter 

In [ ]:
plt.style.use("fivethirtyeight")
winter_copy1 = winter_copy.set_index("time")
winter_copy1.plot(subplots=True, figsize=(20, 10), lw = 0.15)
# plt.plot(dd.time, dd["Temperature (C)"], linewidth=0.5, figsize=(12, 15))

In [ ]:
import statistics
statistics.mean(winter_dataset['Temperature (C)'])

In [ ]:
statistics.mean(winter_dataset['Relative Humidity (%)'])

In [ ]:
statistics.mean(winter_dataset['Pressure (hPa)'])

In [ ]:
import collections
collections.Counter(round(winter_dataset['Temperature (C)']))

In [ ]:
collections.Counter(round(winter_dataset['Relative Humidity (%)']))

In [ ]:
collections.Counter(round(winter_dataset['Pressure (hPa)']))

In [ ]:
# plotting graph
dummy = winter_dataset.copy()
px.scatter_3d(dummy, x='Relative Humidity (%)', y='Temperature (C)', z='Pressure (hPa)', color = dummy['Hour']).update_traces(marker={'size': 2})


In [122]:
# Scalling data
d1 = dummy.copy()
d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]] = StandardScaler().fit_transform(d1.values)

In [123]:
# getting epsilon value
df = dummy
mins = len(df)//300
nn = NearestNeighbors(n_neighbors=mins+1)
nn.fit(d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]])
distances, neighbors = nn.kneighbors(d1[['Hour',"Temperature (C)","Relative Humidity (%)", "Pressure (hPa)"]])

# sort the distances
distances = np.sort(distances[:, mins], axis = 0)

#plot the distances
distances_df = pd.DataFrame({"distances": distances,
                             "index": list(range(0,len(distances)))})

px.line(distances_df, x='index', y='distances')

def calc_distance(x1, y1, a, b, c):
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

def find_eps():
  # (y1 – y2)x + (x2 – x1)y + (x1y2 – x2y1) = 0
  a = distances[0] - distances[-1]  #y
  b = distances_df.index[-1] - distances_df.index[0]    #x
  c1 = distances_df.index[0] * distances[-1]
  c2 = distances_df.index[-1] * distances[0]
  c = c1 - c2

  distance_of_points_from_line = []
  for k in range(len(distances_df)):
    distance_of_points_from_line.append(
        calc_distance(distances_df.index[k], distances[k], a, b, c))
    
  dist = pd.Series(distance_of_points_from_line)
  index_max = dist.idxmax()
  return distances[index_max]

In [124]:
# applying dbscan
db1 = DBSCAN(eps =  find_eps(), min_samples = mins).fit(d1)
labsList = ["Noise"]
labsList = labsList  + ["Cluster " + str(i) for i in range(1,len(set(db1.labels_)))]
d1["assignments"] = db1.labels_
color = d1["assignments"].astype(int)
n_noise_ = list(db1.labels_).count(-1)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of noise points: 116


In [ ]:
# plotting graph with anomalies
px.scatter_3d(dummy, x='Relative Humidity (%)', y='Temperature (C)', z='Pressure (hPa)', color = color).update_traces(marker={'size': dummy['Hour']})

In [127]:
# anomalous data
df1 = winter.copy()
anomaly = d1[d1.assignments == -1]
anomalies_index = list(anomaly.index)
temp_list = []
time_list = []
humidity_list = []
pressure_list = []
# ["Hour","Temperature (C)","Humidity", "Pressure (millibars)"]
for index in anomalies_index:
  temp = df1.loc[index]['Temperature (C)']
  temp_list.append(temp)
  time = df1.loc[index]['time']
  time_list.append(time)
  humidity = df1.loc[index]['Relative Humidity (%)']
  humidity_list.append(humidity)
  pressure = df1.loc[index]['Pressure (hPa)']
  pressure_list.append(pressure)
dic = {
    'Time': time_list,
    'Temperature': temp_list,
    'Humidity': humidity_list,
    'Pressure': pressure_list
}

anomaly_df = pd.DataFrame(dic)

# determining the name of the file
file_name = 'Anomaly_df.csv'
  
# saving the excel
anomaly_df.to_csv(file_name)

anomaly_df


,Time,Temperature,Humidity,Pressure
0,2017-01-06 08:00:00,30.12,74.15,990.27
1,2017-01-06 09:00:00,30.18,73.60,989.62
2,2017-01-06 10:00:00,30.00,74.70,989.14
3,2017-01-06 11:00:00,29.63,77.22,989.00
4,2017-01-06 12:00:00,29.20,80.41,989.26
...,...,...,...,...
111,2019-11-06 20:00:00,27.89,86.38,988.37
112,2019-11-06 21:00:00,27.84,85.81,987.95
113,2019-11-06 22:00:00,27.77,85.65,987.84
114,2019-11-06 23:00:00,27.72,85.69,988.11


In [170]:
winter_temp_anomaly = anomaly_df[['Time','Temperature']]
winter_temp_anomaly1 = winter_temp_anomaly[winter_temp_anomaly['Time'].dt.year == 2017]
winter_temp_anomaly1 = winter_temp_anomaly1[winter_temp_anomaly['Time'].dt.month.isin([2, 1])]
winter_temp = winter[['time',"Temperature (C)"]]
winter_temp1 = winter_temp[winter_temp['time'].dt.year == 2017]
winter_temp1 = winter_temp1[winter_temp['time'].dt.month.isin([2, 1])]

winter_pressure_anomaly = anomaly_df[['Time','Pressure']]
winter_pressure_anomaly1 = winter_pressure_anomaly[winter_pressure_anomaly['Time'].dt.year == 2017]
winter_pressure_anomaly1 =winter_pressure_anomaly1[winter_pressure_anomaly['Time'].dt.month.isin([2, 1])]
winter_pressure = winter[['time',"Pressure (hPa)"]]
winter_pressure1 = winter_pressure[winter_pressure['time'].dt.year == 2017]
winter_pressure1 = winter_pressure1[winter_pressure['time'].dt.month.isin([2, 1])]

winter_humi_anomaly = anomaly_df[['Time','Humidity']]
winter_humi_anomaly1 = winter_humi_anomaly[winter_humi_anomaly['Time'].dt.year == 2017]
winter_humi_anomaly1 = winter_humi_anomaly1[winter_humi_anomaly['Time'].dt.month.isin([2, 1])]
winter_humi = winter[['time',"Relative Humidity (%)"]]
winter_humi1 = winter_humi[winter_humi['time'].dt.year == 2017]
winter_humi1 = winter_humi1[winter_humi['time'].dt.month.isin([2, 1])]


from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1)

#temperature
fig.add_trace(go.Scatter(x=winter_temp1['time'], y=winter_temp1['Temperature (C)'],
                    marker=dict(size=3),
                    mode='markers',
                    name='Temp-lines'),row=1, col=1)
fig.add_trace(go.Scatter(x=winter_temp_anomaly1['Time'], y= winter_temp_anomaly1['Temperature'],
                    marker=dict(size=5),
                    mode='markers',
                    name='anomaly'),row=1, col=1)

#Pressure
fig.add_trace(go.Scatter(x=winter_pressure1['time'], y=winter_pressure1['Pressure (hPa)'],
                    marker=dict(size=3),
                    mode='markers',
                    name='Pressure-lines'),row=2, col=1)
fig.add_trace(go.Scatter(x=winter_pressure_anomaly1['Time'], y= winter_pressure_anomaly1['Pressure'],
                    marker=dict(size=5),
                    mode='markers',
                    name='anomaly'),row=2, col=1)

#humidity
fig.add_trace(go.Scatter(x=winter_humi1['time'], y=winter_humi1['Relative Humidity (%)'],
                    marker=dict(size=3),
                    mode='markers',
                    name='Humidity-lines'),row=3, col=1)
fig.add_trace(go.Scatter(x=winter_humi_anomaly1['Time'], y= winter_humi_anomaly1['Humidity'],
                    marker=dict(size=5),
                    mode='markers',
                    name='anomaly'),row=3, col=1)

fig.update_layout(height=1200, title_text="Anomalies in Temperature, Pressure, Humidity")
# edit axis labels
fig['layout']['xaxis']['title']='Time'
fig['layout']['xaxis2']['title']='Time'
fig['layout']['xaxis3']['title']='Time'
fig['layout']['yaxis']['title']='Temperature'
fig['layout']['yaxis2']['title']='Pressure'
fig['layout']['yaxis3']['title']='Humidity'

fig.show()




In [129]:
dummy['assignments']=d1['assignments']

In [130]:
from sklearn import metrics
metrics.silhouette_score(dummy, dummy['assignments'])

0.09382327090959554